### **Preparação e tratamento dos dados**

___

<br/>

* **Padronização dos nomes das colunas** do dataset para evitar usar muitos *if's* nas etapas posteriores. Dessa forma o CSV vira um contrato para o restante dos módulos

* Foi **utilizada a feature `amount_paid` como valor de referência** para a aresta (transação) para evitar misturar `amount_paid` e `amount_received` em partes diferentes do projeto e não gerar features incoerentes.

* Foram **criadas duas features cambiais**:
   * `fx_spread` que é a diferença entre o valor que saiu e o valor recebido, útil para detectar spreads anômalos e para normalizar/evitar ambiguidades nos cálculos, uma vez que, por exemplo, a transação pode ser paga em BRL e recebida em USD;
   * `currency_pair` que combina as moedas de pagamento e recebimento no formato "origem->destino" e é um forte sinal de padrões suspeitos.

* O dataset foi ordenado por `timestamp` porque se ele estivcer fora de ordem pode acabar tendo vazamentos de informações uma vez que é preciso olhar apenas o histórico passado de uma conta.

* Separação do dataset em train/val/test não por amostragem aleatória das linhas mas por intervalos de data. É uma forma de simular o comportamento em produção de sempre usar o passado para predizer o futuro, além de evitar vazamentos. No fim, essas divisões são salvas em formato `.parquet` após a padronização + limpeza + split dos dados. Dessa forma não é preciso reler todo o CSV bruto, que é muito grande, a cada módulo.

In [ ]:
!pip -q install pandas pyarrow fastparquet pyjanitor datatable pyarrow==17.0.0
!pip -q install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
!pip -q install torch-geometric torch-scatter torch-sparse torch-cluster -f https://data.pyg.org/whl/torch-2.4.0+cu121.html

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

In [4]:
import os, pandas as pd
import numpy as np

In [ ]:
DATA_DIR = '' #colocar em um .env
CSV_PATH = os.path.join(DATA_DIR, 'HI-Small_Trans.csv')

sampled_data = pd.read_csv(CSV_PATH, nrows=5000, low_memory=False)
display(sampled_data.head(3))
print('\n\n')
print(sampled_data.columns.tolist())

In [ ]:
sampled_data.info()

In [7]:
#padronização dos nomes das colunas
RAW_COLS = {
    'Timestamp': 'timestamp',
    'From Bank': 'src_bank',
    'Account': 'src_account',
    'To Bank': 'dst_bank',
    'Account.1': 'dst_account',
    'Amount Received': 'amount_received',
    'Receiving Currency': 'recv_currency',
    'Amount Paid': 'amount_paid',
    'Payment Currency': 'pay_currency',
    'Payment Format': 'payment_format',
    'Is Laundering': 'label',
}

USECOLS = list(RAW_COLS.keys())

In [8]:
#dtypes enxutos pra reduzir memória
DTYPES = {
    'From Bank': 'int32',
    'To Bank': 'int32',
    'Account': 'string',
    'Account.1': 'string',
    'Amount Received': 'float32',
    'Receiving Currency': 'string',
    'Amount Paid': 'float32',
    'Payment Currency': 'string',
    'Payment Format': 'string',
    'Is Laundering': 'int8',
    # 'Timestamp' vem como object e converte pra datetime
}

In [9]:
#leitura em chunks + limpeza
CHUNKSIZE = 500_000
parts = []

for chunk in pd.read_csv(
    CSV_PATH,
    usecols=USECOLS,
    dtype=DTYPES,
    chunksize=CHUNKSIZE,
    low_memory=False
):
    c = chunk.rename(columns=RAW_COLS)

    # timestamp convertido para datetime (no dataset está está "YYYY/MM/DD HH:MM")
    # errors='coerce' garante NaT em linhas ruins
    c['timestamp'] = pd.to_datetime(c['timestamp'], format='%Y/%m/%d %H:%M', errors='coerce', utc=True)

    #remover NaN essenciais
    c = c.dropna(subset=['timestamp', 'src_account', 'dst_account', 'amount_paid'])

    #normalizar strings (evita espaços/lixo)
    for col in ['src_account','dst_account','recv_currency','pay_currency','payment_format']:
        if col in c.columns:
            c[col] = c[col].astype('string').str.strip()

    #amounts negativos/zero: desconsiderados
    c = c[c['amount_paid'] > 0]

    #features úteis:
    # currency_pair
    c['currency_pair'] = (c['pay_currency'].fillna('UNK') + '->' + c['recv_currency'].fillna('UNK')).astype('string')
    #fx_spread (quanto recebeu - quanto pagou)
    c['fx_spread'] = (c['amount_received'].fillna(c['amount_paid']) - c['amount_paid']).astype('float32')

    parts.append(c)

df = pd.concat(parts, ignore_index=True)
del parts

In [10]:
#ordenar por tempo
df = df.sort_values('timestamp').reset_index(drop=True)
df['tx_id'] = np.arange(len(df), dtype=np.int64)

In [ ]:
print(f'shape: {df.shape}\n')
display(df.head(3))
print(f'\n{df.dtypes}\n')
print(f'{df['timestamp'].min()}, "->", {df['timestamp'].max()}')

A página do dataset fala sobre um lote tardio de transações todas ilícitas fora do período principal. Se esse lote cair no teste, não teriam grandes impactos mas se cair no treino, pdoe acabar “ensinando” o modelo com um padrão irreal. Por isso a janela principal foi cortada antes do split, isso foi feito:

* Verificando a séria diária (total vs ilícita)
* Detectando um *tail* no fim que significa os registros ilícitos
* Cortando esse *tail*

Optei por não embaralhar essas linhas no restante dos dados porque em dados financeiros/temporais isso poderia quebrar a causalidade e criar um vazamento. Porém, cortar "cegamente" também pode ser ruim já que eu poderia estar removedo um cenário difícil que seria útil para avaliar a robustez do modelo.

Uma alternativa a essas opções é guardar como um **Tail-ODD** para um teste adicional de robustez sob o desbalanceamento extremo dos dados, como um teste de estresse. Esse conjunto é criado apenas caso esse *tail* seja identificado.

In [12]:
#auditoria diária para achar o tail 100% ilícito
daily = (
    df.assign(day=df['timestamp'].dt.floor('D'))
      .groupby('day', as_index=True)['label']
      .agg(total='size', illicit='sum')
      .assign(all_illicit=lambda x: x['illicit'] == x['total'])
)

#último dia "misto" (tem pelo menos 1 legítima)
mixed_days = daily.index[~daily['all_illicit']]
if len(mixed_days) == 0:
    principal_end = df['timestamp'].max()
else:
    last_mixed_day = mixed_days.max()
    principal_end = last_mixed_day + pd.Timedelta(days=1) - pd.Timedelta(microseconds=1)

In [ ]:
main_df = df[df['timestamp'] <= principal_end].copy()
tail_ood_df  = df[df['timestamp'] >  principal_end].copy()

print("Principal:", main_df['timestamp'].min(), "→", main_df['timestamp'].max(), "|", main_df.shape)
print("Tail-OOD :", tail_ood_df['timestamp'].min() if len(tail_ood_df) else None,
      "→", tail_ood_df['timestamp'].max() if len(tail_ood_df) else None, "|", tail_ood_df.shape)

In [ ]:
ts_min, ts_max = main_df['timestamp'].min(), main_df['timestamp'].max()
t1 = ts_min + (ts_max - ts_min)*0.60
t2 = ts_min + (ts_max - ts_min)*0.80

train_pr = main_df[main_df['timestamp'] <= t1].copy()
valid_pr = main_df[(main_df['timestamp'] > t1) & (main_df['timestamp'] <= t2)].copy()
test_pr  = main_df[main_df['timestamp'] > t2].copy()

print("\nmain dataset splits:")
for name, part in [('train',train_pr),('valid',valid_pr),('test',test_pr)]:
    print(name, part['timestamp'].min(), "→", part['timestamp'].max(), "|", part.shape)

In [15]:
PROC_DIR = os.path.join(DATA_DIR, 'processed'); os.makedirs(PROC_DIR, exist_ok=True)

main_df.to_parquet(os.path.join(PROC_DIR, 'principal_clean.parquet'), index=False)
train_pr.to_parquet(os.path.join(PROC_DIR, 'train_clean.parquet'), index=False)
valid_pr.to_parquet(os.path.join(PROC_DIR, 'valid_clean.parquet'), index=False)
test_pr .to_parquet(os.path.join(PROC_DIR, 'test_clean.parquet' ), index=False)

if len(tail_ood_df):
    tail_ood_df.to_parquet(os.path.join(PROC_DIR, 'tail_ood_clean.parquet'), index=False)